# Aprendizaje semisupervisado

Durante este cuadernillo veremos la estrategia de aprendizaje semisupervisado. Realmente, esta estrategia se fusiona con la de aprendizaje supervisado. No tiene modelos específicos por sí misma, sino que utiliza todo lo que ya hemos visto hasta ahora.

_No he añadido las siglas de la etiqueta por razones evidentes: si aprendizaje no supervisado era ANS y aprendizaje supervisado era AS, aprendizaje semisupervisado solo podía ser..._

## Introducción teórica

Hemos visto que las técnicas de aprendizaje supervisado dan resultados muy buenos. Sin embargo, es cierto que trabajar siempre con todos los datos etiquetados puede ser, más o menos, algo utópico.

En la vida real, las etiquetas de los datos son obtenidas por procesos generalmente costosos: ya sea porque sean caras de conseguir (sensores) o porque se necesite contratar a gente muy experta para realizar una tarea de clasificación (médicos para detectar posibles infartos, por ejemplo). Por ello, no siempre es fácil obtener las etiquetas de nuestro conjunto de datos. Para solventar este problema, se creó el aprendizaje semisupervisado.

La idea principal de este tipo de aprendizaje es que partimos de un conjunto de datos que **NO** tiene todas las etiquetas que necesitamos (ya sea porque sean entradas con nulos en una tabla o porque sean imágenes sin etiqueta, como veremos en el tema 4). Para convertir este tipo de situaciones en una situación de aprendizaje supervisado (que ya sabemos cómo tratar), crearemos un modelo capaz de etiquetarse a sí mismo los datos.

El proceso, por tanto, debe ser cuidadoso: no podemos hacerlo de un solo golpe, dado que el modelo podría tener dudas con algunos datos muy particulares, pero muy posiblemente sí tengamos datos que indiscutiblemente tengan una etiqueta particular. Por ello, es evidente que el proceso de etiquetado debe ser iterativo. Seguiremos la siguiente hoja de ruta:

1. Entrenaremos un modelo con todos los datos que tenemos etiquetados.
2. Realizaremos la predicción para todos los datos para los que **NO** tengamos una etiqueta.
3. Aquellos datos que tengan una probabilidad superior a un _threshold_ de estar bien etiquetados (veremos cómo podemos obtener este dato) les asignaremos esa etiqueta.
4. Repetiremos este proceso un número determinado de veces.

## Conjunto de datos

Durante este cuadernillo, trabajaremos con el conjunto de datos del Titanic.

Como este conjunto está completo, eliminaremos parte de la información para poder simular una situación en la que usaríamos aprendizaje semisupervisado.

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import math

In [ ]:
df = sns.load_dataset("titanic", cache=False)
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


Separo mi conjunto de testeo.

In [ ]:
from sklearn.model_selection import train_test_split

random_seed = 33

train, test = train_test_split(df, test_size=0.2, random_state=random_seed)

Vamos a predecir si la persona sobrevivió o no.

Realizamos la limpieza de los datos.

Elimino todas las columnas que no voy a utilizar.

In [ ]:
train = train.drop(columns=["pclass", "sibsp", "parch", "embarked", "who", "adult_male", "deck", "alive"])
train.head()

,survived,sex,age,fare,class,embark_town,alone
461,0,male,34.0,8.0500,Third,Southampton,True
670,1,female,40.0,39.0000,Second,Southampton,False
877,0,male,19.0,7.8958,Third,Southampton,True
664,1,male,20.0,7.9250,Third,Southampton,False
44,1,female,19.0,7.8792,Third,Queenstown,True


¿Tengo nulos o atípicos?

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 461 to 20
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     712 non-null    int64   
 1   sex          712 non-null    object  
 2   age          566 non-null    float64 
 3   fare         712 non-null    float64 
 4   class        712 non-null    category
 5   embark_town  710 non-null    object  
 6   alone        712 non-null    bool    
dtypes: bool(1), category(1), float64(2), int64(1), object(2)
memory usage: 34.9+ KB


In [ ]:
train.describe()

,survived,age,fare
count,712.000000,566.000000,712.000000
mean,0.379213,29.789170,30.949267
std,0.485532,14.205749,49.616295
min,0.000000,0.420000,0.000000
25%,0.000000,21.000000,7.895800
50%,0.000000,28.000000,14.254150
75%,1.000000,38.000000,30.000000
max,1.000000,80.000000,512.329200


Elimino los nulos de ```embark_town```. Para tratar los de ```age``` (quiero predecirlos) necesito limpiar el conjunto de datos, así que los dejo por ahora. Fíjate que este proceso es un subproceso de aprendizaje semisupervisado.

In [ ]:
train["embark_town"] = train["embark_town"].apply(lambda e: str(e))
train = train[train.embark_town != "nan"]
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 710 entries, 461 to 20
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     710 non-null    int64   
 1   sex          710 non-null    object  
 2   age          564 non-null    float64 
 3   fare         710 non-null    float64 
 4   class        710 non-null    category
 5   embark_town  710 non-null    object  
 6   alone        710 non-null    bool    
dtypes: bool(1), category(1), float64(2), int64(1), object(2)
memory usage: 34.8+ KB


Elimino mis datos atípicos.

In [ ]:
def outlier_eliminator(df, threshold: float = 0.05):
  s_df = df.describe() # statistical dataframe

  for column in s_df.columns:
    # datos necesarios
    n = s_df.loc["count", column]
    q1 = s_df.loc["25%", column]
    q3 = s_df.loc["75%", column]
    mean = s_df.loc["mean", column] # 50% no es la media, es la mediana
    iqr = (q3 - q1) * 1.5

    # calculos
    _range = [mean - iqr, mean + iqr]
    outliers = df[(df[column] < _range[0]) | (df[column] > _range[1])]

    # outliers
    print(f"Outliers para la columna {column}: {len(outliers)} de {n}. BORRADOS: ", end="")
    if len(outliers) != 0 and len(outliers) / n < threshold:
      print("Sí.")
      df = df[(df[column] > _range[0]) & (df[column] < _range[1])] # fíjate que le doy la vuelta
    else:
      print("No.")

    print("### --- ###")

  return df

In [ ]:
train = outlier_eliminator(train)

Outliers para la columna survived: 0 de 710.0. BORRADOS: No.
### --- ###
Outliers para la columna age: 60 de 564.0. BORRADOS: No.
### --- ###
Outliers para la columna fare: 84 de 710.0. BORRADOS: No.
### --- ###


Tengo muchos, así que eliminarlos podría empeorar mi modelo.

Codifico mis columnas categóricas.

In [ ]:
train.sex = train.sex.apply(lambda v: 0 if v == "male" else 1 if v == "female" else v)
train.head()

,survived,sex,age,fare,class,embark_town,alone
461,0,0,34.0,8.0500,Third,Southampton,True
670,1,1,40.0,39.0000,Second,Southampton,False
877,0,0,19.0,7.8958,Third,Southampton,True
664,1,0,20.0,7.9250,Third,Southampton,False
44,1,1,19.0,7.8792,Third,Queenstown,True


In [ ]:
train["class"] = train["class"].apply(lambda v: 1 if v == "First" else 2 if v == "Second" else 3 if v == "Third" else v)
train["class"] = train["class"].astype(int)
train.head()

,survived,sex,age,fare,class,embark_town,alone
461,0,0,34.0,8.0500,3,Southampton,True
670,1,1,40.0,39.0000,2,Southampton,False
877,0,0,19.0,7.8958,3,Southampton,True
664,1,0,20.0,7.9250,3,Southampton,False
44,1,1,19.0,7.8792,3,Queenstown,True


In [ ]:
train["alone"] = train["alone"].apply(lambda v: int(v))
train.head()

,survived,sex,age,fare,class,embark_town,alone
461,0,0,34.0,8.0500,3,Southampton,1
670,1,1,40.0,39.0000,2,Southampton,0
877,0,0,19.0,7.8958,3,Southampton,1
664,1,0,20.0,7.9250,3,Southampton,0
44,1,1,19.0,7.8792,3,Queenstown,1


In [ ]:
def binary_categorizer(dataframe, column, code_map: dict = None, cols: int = None):
  result = [] # resultados
  if not cols: # puede ser que me obliguen a que haya un número determinado de columnas
    cols = math.ceil(math.log2(len(dataframe[column].unique()))) # aplico la fórmula de log_2_n y lo aproximo al número más grande
  if not code_map:
    code_map = {value: key for key, value in enumerate(dataframe[column].unique())} # creo el mapa de forma genérica si no existe

  for value in dataframe[column]: # para cada valor
    code = code_map[value] # recojo el código asignado
    b_code = format(code, "b") # lo convierto a binario

    if len(b_code) > cols:
      raise Exception(f"El número de columnas ({cols}) es demasiado pequeño para empaquetar la información ({len(b_code)}). Modifica el valor del atributo cols.")

    b_code_a = b_code.rjust(cols, "0") # lo formateo hasta tamaño cols rellenando con 0
    _value = list(b_code_a) # lo convierto a lista: cada elemento en una posición diferente 00 -> ["0", "0"]
    result.append(list(map(lambda v: int(v), _value))) # convierto la lista en una lista de enteros ["0", "0"] -> [0, 0]

  new_columns_name = [f"{column}_{i}" for i in range(len(list(result[0])))] # les daré nombre a las nuevas columnas
  result_df = pd.DataFrame(result, index=dataframe.index, columns=new_columns_name) # creo un nuevo df con los resultados
  dataframe = pd.concat([dataframe, result_df], axis=1) # lo añado en el eje X respetando el orden
  return dataframe.drop(columns=[column]), code_map

In [ ]:
train, embark_map_code = binary_categorizer(train, "embark_town")
train.head()

,survived,sex,age,fare,class,alone,embark_town_0,embark_town_1
461,0,0,34.0,8.0500,3,1,0,0
670,1,1,40.0,39.0000,2,0,0,0
877,0,0,19.0,7.8958,3,1,0,0
664,1,0,20.0,7.9250,3,0,0,0
44,1,1,19.0,7.8792,3,1,0,1


Voy a predecir los valores de ```age```. Separo el conjunto en datos conocidos y desconocidos.

In [ ]:
known_data = train[~train.age.isnull()]
unknown_data = train[train.age.isnull()]

Separo mi variable a predecir: la variable ```age```.

In [ ]:
X_known_data, y_known_data = known_data.drop(columns="age"), known_data["age"]
X_unknown_data = unknown_data.drop(columns="age") # el valor de la Y aquí me da igual, todas valen Unknown

```age``` es una variable numérica (siempre hemos hablado sobre si era categórica o numérica. En este problema no tenemos suficientes datos como para tratarla como categórica). Creo un regresor. Por ejemplo usando un árbol de decisión.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
age_model = DecisionTreeRegressor() # podría optimizar este modelo, pero no lo voy a hacer
age_model = age_model.fit(X_known_data, y_known_data)
age_preds = age_model.predict(X_unknown_data)
age_preds

array([39.        , 40.        , 21.        , 19.        , 23.        ,
       24.        , 19.        , 19.        , 27.11764706, 40.        ,
       20.        , 26.4       , 43.5       ,  1.        , 43.5       ,
       35.3       , 27.11764706, 19.        , 35.        , 28.        ,
       40.        , 43.5       , 23.        , 41.        ,  8.75      ,
       35.        , 34.        , 43.5       , 28.5       , 20.        ,
        4.        , 27.11764706, 43.5       , 80.        , 30.8125    ,
       28.5       ,  4.        , 43.5       , 33.92307692, 15.        ,
       30.        , 36.5       , 22.        , 33.92307692, 28.        ,
       28.        , 19.        , 19.        , 19.        , 27.11764706,
       43.        , 25.        , 29.71428571, 41.        , 45.        ,
       27.11764706, 19.        , 19.        , 31.97222222, 25.83333333,
       43.5       , 47.        , 26.        , 31.        , 19.        ,
       21.        , 33.92307692, 57.6       , 30.8125    , 31.97

In [ ]:
unknown_data.loc[:, "age"] = age_preds
train = pd.concat([known_data, unknown_data])
train.head()

,survived,sex,age,fare,class,alone,embark_town_0,embark_town_1
461,0,0,34.0,8.0500,3,1,0,0
670,1,1,40.0,39.0000,2,0,0,0
877,0,0,19.0,7.8958,3,1,0,0
664,1,0,20.0,7.9250,3,0,0,0
44,1,1,19.0,7.8792,3,1,0,1


**MUY IMPORTANTE:** mezclamos los datos para no tenerlos ordenados.

In [ ]:
train = train.sample(frac=1) # mezclo el 100% de los datos
train.head()

,survived,sex,age,fare,class,alone,embark_town_0,embark_town_1
288,1,0,42.000000,13.0,2,1,0,0
616,0,0,34.000000,14.4,3,0,0,0
277,0,0,33.923077,0.0,2,1,0,0
17,1,0,36.500000,13.0,2,1,0,0
766,0,0,36.000000,39.6,1,1,1,0


## Aplicando el aprendizaje semisupervisado

Tras un problema en la base de datos, perdemos el 95% de las etiquetas:

In [ ]:
import random

train["survived"] = train["survived"].apply(lambda v: None if random.random() < 0.95 else v)
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 710 entries, 288 to 786
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   survived       34 non-null     float64
 1   sex            710 non-null    int64  
 2   age            710 non-null    float64
 3   fare           710 non-null    float64
 4   class          710 non-null    int64  
 5   alone          710 non-null    int64  
 6   embark_town_0  710 non-null    int64  
 7   embark_town_1  710 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 49.9 KB


Ahora comienza el proceso iterativo: creamos una función para aprendizaje semisupervisado.

Dentro de esta función, usaremos la función ```predict_proba```. Esta función devuelve una lista de probabilidades para cada clase de la clasificación en la que se posee la probabilidad de que pertenezca a cada una de las clases. La clase seleccionada, usando ```predict```, es aquella que sea más probable. Sin embargo, ahora no nos interesa solo la clase, sino también la probabilidad de que sea realmente esa clase. Como usar un _threshold_ fijo me podría ocasionar un bucle infinito, voy a ir reduciendo mi restricción cada vez que haga una iteración sin mejorar mi cantidad de datos desconocidos.

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def semisupervised(df, target, threshold = 0.9, max_iter = 10, max_iter_same_nulls = 5, strategy = "iterative", regression = False):
  # general
  iter = 1

  # strategy = fixed
  nulls = 0
  iter_same_nulls = 0
  known_data = df.dropna() # los únicos nulos son los de mi predicción
  unknown_data = df.loc[list(set(df.index) - set(known_data.index)), :] # lo hago así para evitar problemas de nulos
  while True:
    if unknown_data.empty:
      print("Me he quedado sin nulos.")
      break

    elif strategy == "fixed" and iter_same_nulls >= max_iter_same_nulls:
      print("La estrategia es 'fixed' y he superado mi límite de iteraciones.")
      break

    elif iter >= max_iter and max_iter != -1:
      print("He superado mi límite de iteraciones")
      break

    nulls = len(unknown_data)
    print(f"Iteración {iter}. Cantidad de nulos antes del proceso en la columna '{target}': {nulls}.")

    ## extracción de la X y la Y
    X_known_data, y_known_data = known_data.drop(columns=target), known_data[target]
    if not regression: y_known_data = y_known_data.apply(lambda v: str(int(v))) # lo convierto a cadenas, porque al tener nulos eran reales y necesito clases
    X_unknown_data = unknown_data.drop(columns=target) # el valor de la Y aquí me da igual, todas valen Unknown

    # creación de un modelo de clasificación (fíjate, survived es una categoría binaria)
    model = KNeighborsClassifier()
    model = model.fit(X_known_data, y_known_data)
    preds_proba = model.predict_proba(X_unknown_data)

    # predicciones admitidas
    # * la función np.argmax devuelve el índice con el valor más grande dentro de un iterable
    preds = [np.argmax(pred) if pred[np.argmax(pred)] >= threshold else None for pred in preds_proba]
    unknown_data[target] = preds

    # obtengo los datos que ahora SÍ conozco (para evitar un FutureWarning)
    now_known_data = unknown_data.dropna()

    if not now_known_data.empty:
      known_data = pd.concat([known_data, now_known_data])

    # me quedo solo con los datos que no conozco
    unknown_data = unknown_data[unknown_data[target].isna()]

    if len(unknown_data) == nulls:
      iter_same_nulls += 1
      if strategy == "iterative":
        threshold -= 0.01 # estrategia iterativa: si no puedo avanzar, soy un poco más laxo
    else:
      iter_same_nulls = 0

    iter += 1

  if unknown_data.empty:
    df = known_data
  else:
    df = pd.concat([known_data, unknown_data])

  return df.sample(frac=1) # mezclamos los datos

In [ ]:
train = semisupervised(train, "survived", max_iter = -1)

Iteración 1. Cantidad de nulos antes del proceso en la columna 'survived': 676.
Iteración 2. Cantidad de nulos antes del proceso en la columna 'survived': 476.
Iteración 3. Cantidad de nulos antes del proceso en la columna 'survived': 328.
Iteración 4. Cantidad de nulos antes del proceso en la columna 'survived': 256.
Iteración 5. Cantidad de nulos antes del proceso en la columna 'survived': 224.
Iteración 6. Cantidad de nulos antes del proceso en la columna 'survived': 198.
Iteración 7. Cantidad de nulos antes del proceso en la columna 'survived': 177.
Iteración 8. Cantidad de nulos antes del proceso en la columna 'survived': 168.
Iteración 9. Cantidad de nulos antes del proceso en la columna 'survived': 165.
Iteración 10. Cantidad de nulos antes del proceso en la columna 'survived': 163.
Iteración 11. Cantidad de nulos antes del proceso en la columna 'survived': 163.
Iteración 12. Cantidad de nulos antes del proceso en la columna 'survived': 163.
Iteración 13. Cantidad de nulos antes

Convierto las etiquetas en enteros, porque como había nulos pueden ser decimales.

In [ ]:
train["survived"] = train.survived.apply(lambda v: str(int(v)))
train.survived.isna().any() # ¿hay algún nulo?

np.False_

## Aprendizaje supervisado

Ahora que ya tenemos todas las etiquetas, podemos entrenar un modelo clásico de los del grupo de aprendizaje supervisado.

Separamos la X y la Y. No haré validación ni optmización, pero podríamos hacerlo.

In [ ]:
X_train, y_train = train.drop(columns="survived"), train.survived

Obtengo mis predicciones.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model = MLPClassifier(random_state=random_seed)
model = model.fit(X_train, y_train)
preds = model.predict(X_train)
accuracy_score(y_train, preds)

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9788732394366197

Para entrenamiento, el resultado es muy bueno.

Esto es común al usar estrategias semisupervisadas. Veamos a ver si esto sigue siendo así con el testeo.

Recuperamos el conjunto de test y aplicamos el mismo proceso de limpieza.

In [ ]:
test

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
297,0,1,female,2.0,1,2,151.5500,S,First,child,False,C,Southampton,no,False
71,0,3,female,16.0,5,2,46.9000,S,Third,woman,False,NaN,Southampton,no,False
631,0,3,male,51.0,0,0,7.0542,S,Third,man,True,NaN,Southampton,no,True
567,0,3,female,29.0,0,4,21.0750,S,Third,woman,False,NaN,Southampton,no,False
21,1,2,male,34.0,0,0,13.0000,S,Second,man,True,D,Southampton,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,0,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
721,0,3,male,17.0,1,0,7.0542,S,Third,man,True,NaN,Southampton,no,False
286,1,3,male,30.0,0,0,9.5000,S,Third,man,True,NaN,Southampton,yes,True
759,1,1,female,33.0,0,0,86.5000,S,First,woman,False,B,Southampton,yes,True


In [ ]:
# columnas
test = test.drop(columns=["pclass", "sibsp", "parch", "embarked", "who", "adult_male", "deck", "alive"])

# categorizacion de variables
test.sex = test.sex.apply(lambda v: 0 if v == "male" else 1 if v == "female" else v)
test["class"] = test["class"].apply(lambda v: 3 if v == "First" else 2 if v == "Second" else 1 if v == "Third" else v)
test["class"] = test["class"].astype(int)
test["alone"] = test["alone"].apply(lambda v: 1 if v else 0)
test, _ = binary_categorizer(test, "embark_town", code_map=embark_map_code)

# valores de age (USO EL MISMO PREDICTOR QUE PARA EL ENTRENAMIENTO)
test["age"] = test["age"].apply(lambda e: "Unknown" if str(e) == "nan" else e)
known_data = test[test.age != "Unknown"]
unknown_data = test[test.age == "Unknown"].copy()
X_unknown_data = unknown_data.drop(columns="age")
age_preds = age_model.predict(X_unknown_data)
unknown_data["age"] = age_preds
test = pd.concat([known_data, unknown_data])
test = test.sample(frac=1) # los mezclo!
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 300 to 22
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   survived       179 non-null    int64  
 1   sex            179 non-null    int64  
 2   age            179 non-null    object 
 3   fare           179 non-null    float64
 4   class          179 non-null    int64  
 5   alone          179 non-null    int64  
 6   embark_town_0  179 non-null    int64  
 7   embark_town_1  179 non-null    int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 12.6+ KB


Predigo mi testeo.

In [ ]:
X_test, y_test = test.drop(columns="survived"), test.survived.apply(lambda v: str(int(v)))

In [ ]:
preds = model.predict(X_test)

In [ ]:
accuracy_score(y_test, preds)

0.6089385474860335

Como podemos ver, la predicción es bastante peor que la que teníamos al usar las etiquetas reales. Pero fíjate lo que hemos hecho: hemos logrado un clasificador mejor que la media usando un 5% de los datos (el resto han sido etiquetados de forma artificial, **SIN AYUDA DE UN EXPERTO**).

# Resumen

Durante este cuadernillo hemos visto cómo aplicar técnicas de aprendizaje semisupervisado para predecir las etiquetas no disponibles durante el entrenamiento. Además, hemos visto también como predecir valores nulos (que lo dejamos pendiente en el tema 2).

Esta técnica suele usarse con frecuencia en problemas donde conseguir las etiquetas es un proceso muy costoso (en tiempo o en dinero).

Hay que tener cuidado al usar esta estrategia, pues suele inducir a sobrentrenamientos.